### Try to use LinearMap with ismutating=true, fail

In [ ]:
using LinearMaps
using IterativeSolvers

In [ ]:
function g(y::AbstractVector,x::AbstractVector) 
    y = [1 2;3 5]*x
    return y
end

S = LinearMap(g,2;ismutating=true)
b = [4.0, 3.3]
x = [2.0, 2.0]
gmres!(x,S,b)

### Try to use LinearMap with ismutating=false, works

In [ ]:
function f(x::AbstractVector) 
    return [1 2;3 5]*x
end

S = LinearMap(f,2;ismutating=false)
b = [4.0, 3.3]
gmres(S,b)

In [ ]:
[1 2;3 5]\[4.,3.3]

### Construct block matrix with functions, works

In [ ]:
m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]

[m1 m2;m3 m4]\b

In [ ]:
function al(x::AbstractVector)

m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]

    
    function f(x::AbstractVector) 
        return m1*x
    end
    function g(x::AbstractVector) 
        return m2*x
    end
    function m(x::AbstractVector) 
        return m3*x
    end
    function n(x::AbstractVector) 
        return m4*x
    end 
    
    # has to do the matrix mul manually to return 1d output
    out = zeros(5)
    out[1:3] = f(x[1:3]) + g(x[4:5])
    out[4:5] = m(x[1:3]) + n(x[4:5])
    
    return out
end
S = LinearMap(al,5;ismutating=false)    
    
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]
gmres(S,b)

### Test SaddleSystem

In [7]:
A = rand(80,80)
A⁻¹ = inv(A)
B₁ᵀ = rand(80,12)
B₂ = rand(12,80)
M = rand(12,12)
M⁻¹ = inv(M)
G₁ᵀ = rand(12,12)
G₂ = rand(12,12)
T₁ᵀ = rand(12,12)
T₂ = rand(12,12)

O32 = zeros(80,12)
O23 = zeros(12,80)
O22 = zeros(12,12)

S = [A O32 B₁ᵀ O32; O23 M T₁ᵀ G₁ᵀ; B₂ -T₂ O22 O22; O23 G₂ O22 O22]

rċ = rand(80)
ru̇ = rand(12)
rf = rand(12)
rλ = rand(12)
b = [rċ;ru̇;rf;rλ]

S\b

116-element Array{Float64,1}:
  10.9637 
  -4.19658
  -1.49447
   1.15622
  -1.4213 
  10.6051 
   5.11822
  -1.10456
  -8.40552
  -6.19242
  -2.77369
   6.09046
   7.9735 
   ⋮      
 -25.77   
 -40.048  
  61.0844 
 -35.9778 
  19.4345 
 -23.011  
  38.4719 
  42.4842 
 -50.1064 
  38.2913 
 -35.7336 
  11.2997 

In [8]:
norm(S*(S\b)-b)

2.3833120458742235e-13

In [9]:
include("../src/SaddlePointSystems.jl")
using SaddlePointSystems

ċ = zeros(80)
u̇ = zeros(12)
f = zeros(12)
λ = zeros(12)

St = SaddleSystem((ċ, u̇, f, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M⁻¹, G₁ᵀ, G₂), (T₁ᵀ, T₂))

bt = (rċ, ru̇, rf, rλ);

In [10]:
aa,bb,cc,dd = St\bt

([0.369329, -0.697804, 0.732585, -0.201039, 0.641821, 0.576867, -0.19927, -0.0400248, -0.116001, -0.177289  …  0.644777, 0.839327, 0.266865, -0.294075, -0.72746, -0.704247, -0.114817, -1.1125, 0.728516, -0.912563], [0.212307, -0.605037, -0.647972, 0.502702, 0.603824, 0.655128, 0.380829, -0.361744, 0.100293, -0.443483, 0.809765, 0.683713], [-0.551856, -0.197221, 0.531107, 0.5343, 0.111864, -0.720829, -0.620866, -0.994125, 1.06497, -0.352837, -0.481323, 0.0834559], [0.162369, -0.499748, -0.221312, 0.546005, -0.16729, 1.70887, 0.726356, -0.468532, -0.953857, -0.330789, -0.54405, 0.969441])

In [11]:
x = [aa;bb;cc;dd]
norm(S*x-b)

0.5542617837526679